# 2. Clair と Klar で Docker イメージを脆弱性スキャン

実際に脆弱性スキャンしてみましょう！

## 2.1. ソフトウェアとそのアーキテクチャについて

### Clair

Clair は CoreOS 社の Docker イメージ脆弱性情報管理ツールです。  
**脆弱性スキャンそのもの** と **脆弱性データベースを管理する** API を持っており、  
脆弱性データベースそのものは PostgreSQL を内部データベースとして管理されます。  
https://github.com/coreos/clair

後半 Trivy といった似通ったツールを紹介していますが、  
それと比較して Clair を使うべきシチュエーションは例えば [ベース OS のサポート](https://github.com/coreos/clair/blob/master/Documentation/drivers-and-data-sources.md#data-sources-for-the-built-in-drivers) であったりします。

### Klar

**Clair の API を内部的に利用**し、  
任意の Docker Registry にある Docker イメージの脆弱性が検査できます。  
Clair の API だけでは少々使い勝手が悪いところを解消してくれます。  
https://github.com/optiopay/klar

### AWS 公式にある構成事例

- 昨年末 [Scanning Docker Images for Vulnerabilities using Clair, Amazon ECS, ECR, and AWS CodePipeline](https://aws.amazon.com/blogs/compute/scanning-docker-images-for-vulnerabilities-using-clair-amazon-ecs-ecr-aws-codepipeline/)
- 先週 [Detect vulnerabilities in the Docker images in your applications](https://aws.amazon.com/jp/blogs/publicsector/detect-vulnerabilities-in-the-docker-images-in-your-applications/)

## 2.2. ツールをすべて Dockerize

### Clair

AWS 公式の方法は PostgreSQL に [Amazon Aurora](https://aws.amazon.com/jp/rds/aurora/) を使っていますが、ここに可用性をどこまで求めるかはプロジェクト次第かと思います。例えば脆弱性データベースを作り直すことは難しくはないので、その間 False Negative な結果を返してもいいかなど。そこで今回はコンテナ支部らしく、ECS だけで Clair を利用してみます。

1. まず Clair そのものを例えば [このような Dockerfile](https://github.com/supinf/dockerized-tools/blob/master/golang/clair/versions/2.0/Dockerfile) でビルドし
2. [このような Dockerfile](https://github.com/supinf/dockerized-tools/blob/master/golang/clair/with-db/versions/2.0/Dockerfile) でデータベースを同じコンテナに押し込んでしまいます

これは複数プロセスをひとつのコンテナに入れているためアンチパターンとも言える構成ですが、そもそも可用性を求めていないのと、こうすることでローカルからでも `docker run` で気軽に起動できるといったメリットもあります。

### Klar

同様に Klar もコンテナ化します。  
ECR のイメージでも簡単に動作するようちょっと工夫してしまっていますが、例えば [このような Dockerfile](https://github.com/supinf/dockerized-tools/blob/master/golang/klar/versions/2.4/Dockerfile) で。

## 2.3. 一般 Docker イメージのスキャン

スキャンを実行してみます。

In [ ]:
source ~/config/.env

docker run --rm \
      -e CLAIR_ADDR="http://${API_HOST}:6060" \
      supinf/klar:2.4 envoyproxy/envoy-alpine:v1.10.0 | jq .

成功時応答）
```text
{
  "LayerCount": 6,
  "Vulnerabilities": {}
}
```

脆弱性をもつ Docker イメージで、緊急度を **Medium** にして試してみましょう。

In [ ]:
docker run --rm \
      -e CLAIR_ADDR="http://${API_HOST}:6060" -e CLAIR_OUTPUT=Medium \
      supinf/klar:2.4 envoyproxy/envoy:v1.10.0 | jq .

成功時応答例）
```text
{
  "LayerCount": 9,
  "Vulnerabilities": {
    "Medium": [
      {
        "Name": "CVE-2009-5155",
        "NamespaceName": "ubuntu:16.04",
        "Description": "In the GNU C Library (aka glibc or libc6) before 2.28, .."
        ..
      },
      {
        "Name": "CVE-2019-9893",
        "NamespaceName": "ubuntu:16.04",
        "Description": "libseccomp before 2.4.0 ..",
        ..
      }
    ]
  }
}
```

**脆弱性 0 件でしたか？？**  
とすると、それは脆弱性データベースがバックエンドで作成中だからです。  
しばらくしてから再度試してみてください。

## 2.4. ECR においた Docker イメージのスキャン

ECR へ任意の Docker イメージでもいいのですが、今回はこのハンズオンイメージを push し  
それに対してスキャンをかけてみます。

In [ ]:
image_name="$(aws sts get-caller-identity --query "Account" --output text).dkr.ecr.${AWS_DEFAULT_REGION}.amazonaws.com/jawsug/container:scan-images"
docker tag jawsug/container:scan-images "${image_name}"
aws ecr create-repository --repository-name jawsug/container
$( aws ecr get-login --no-include-email )
docker push "${image_name}"

いま push したイメージにスキャンをかけてみます。

In [ ]:
docker run --rm \
      -e AWS_ACCESS_KEY_ID -e AWS_SECRET_ACCESS_KEY -e AWS_DEFAULT_REGION \
      -e CLAIR_ADDR="http://${API_HOST}:6060" -e CLAIR_OUTPUT=Low \
      supinf/klar:2.4 ${image_name} | jq .

成功時応答例）
```text
{
  "LayerCount": 20,
  "Vulnerabilities": {
    "High": [
      {
        "Name": "CVE-2016-4074",
        "NamespaceName": "alpine:v3.9",
        ..
      }
    ]
  }
}
```

[次へ: 03-trivy](03-trivy.ipynb)